In [1]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [2]:
from bs4 import BeautifulSoup
import requests
import json

In [8]:
condition = input("please input 1 to proccess on one URL or 2 to proccess on a search URL")
URL = input("please input the URL")
webpage = requests.get(URL, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")

please input 1 to proccess on one URL or 2 to proccess on a search URL1
please input the URLhttps://www.amazon.com/PlayStation-4-Console-1TB-Slim/dp/B074LRF639/ref=sr_1_1?dchild=1&keywords=playstation+4&qid=1610625433&sr=8-1


In [9]:
def webScrapping(soup,File):
    try:
        title = soup.find("span", attrs={"id":'productTitle'})
        title_value = title.string
        title_string = title_value.strip().replace(',','')
    except:
        title_string = "N/A"
    print("Product Title = ", title_string)
     
    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip().replace(',','')
    except:
        price = "N/A"
    print("product Price = ",price)
     
    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip().replace(',','')
    except:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip().replace(',','')
        except:
            rating = "N/A"
    print("product rating = ",rating)
    if(rating == "4 Stars & Up"):
        return -1
   
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip().replace(',','')
    except:
        review_count = "N/A"
    print("product rating count = ",review_count)
    
    try:
        company = soup.find('a',id='sellerProfileTriggerId').get_text()
    except:
        company="N/A"
    print("the company that sells = ",company)
    
    img_div = soup.find(id="imgTagWrapperId")

    imgs_str = img_div.img.get('data-a-dynamic-image')  # a string in Json format

    # convert to a dictionary
    imgs_dict = json.loads(imgs_str)
    #each key in the dictionary is a link of an image, and the value shows the size (print all the dictionay to inspect)
    num_element = 0 
    first_link = list(imgs_dict.keys())[num_element]
    print("product image URL = ",first_link)
    File.write(f"{title_string},")
    File.write(f"{price},")
    File.write(f"{rating},")
    File.write(f"{review_count},")
    File.write(f"{company},")
    File.write(f"{first_link},\n")


In [10]:
import os
if os.path.exists("out.csv"):
    os.remove("out.csv")
else:
    print("The file does not exist")
File = open("out.csv", "a")
File.write(f"Product Title,")
File.write(f"product Price,")
File.write(f"product rating,")
File.write(f"product rating count,")
File.write(f"the company that sells,")
File.write(f"product image URL,\n")
if(condition == "1"):
    webScrapping(soup,File)
elif(condition == "2"):
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})
    links_list = []
    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    for link in links_list:
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
            new_soup = BeautifulSoup(new_webpage.content, "lxml")
            webScrapping(new_soup,File)
File.close()

Product Title =  PlayStation 4 Console - 1TB Slim Edition
product Price =  $419.99
product rating =  4.8 out of 5 stars
product rating count =  3553 ratings
the company that sells =  TRENDY-TECH (We Record Serial Numbers)
product image URL =  https://images-na.ssl-images-amazon.com/images/I/715RBdgZtHL._SX425_.jpg
